---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [4]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def graph_identification():
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

#for i in range(5):
#    c = nx.average_clustering(P1_Graphs[i])
#    print(f'Avg Clustering Coef of {i} is {c}.')
#print('\n')

#for i in range(5):
#    l = nx.average_shortest_path_length(P1_Graphs[i])
#    print(f'Avg Distance of {i} is {l}.')
#print('\n')

#for i in range(5):
#    n_nodes = nx.number_of_nodes(P1_Graphs[i])
#    print(f'Graph {i} has {n_nodes} nodes.')
    
#import matplotlib.pyplot as plt

#for i in range(5):
#    fig = plt.figure()
#    plt.title(f'Graph {i}')
#    degrees = P1_Graphs[i].degree()
#    degree_values = sorted(set(dict(degrees).values()))
#    n_nodes = float(nx.number_of_nodes(P1_Graphs[i]))
#    histogram = [list(dict(degrees).values()).count(i) / n_nodes for i in degree_values]
#    plt.bar(degree_values, histogram)
#    plt.show()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

AttributeError: 'Graph' object has no attribute '_node'

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
def salary_predictions():
    # Creating a DataFrame with the features:
    # Deparment, Degree, Closeness Centrality, Betweenness Centrality and Clustering
    # and also a column with the Management Salary
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Degree'] = pd.Series(G.degree())
    # DegreeCentrality and Degree have a high correlation
    #df['DegreeCentrality'] = pd.Series(nx.degree_centrality(G))
    df['ClosenessCentrality'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['BetweennessCentrality'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df['Clustering'] = pd.Series(nx.clustering(G))

    # Diving into the training and test DataFrames
    df_test = df[np.isnan(df['ManagementSalary'])]
    df_train = df[~np.isnan(df['ManagementSalary'])]

    X_train = df_train[['Department', 'Degree', 'ClosenessCentrality', 'BetweennessCentrality', 'Clustering']]
    y_train = df_train['ManagementSalary']
    X_test = df_test[['Department', 'Degree', 'ClosenessCentrality', 'BetweennessCentrality', 'Clustering']]

    from sklearn.ensemble import GradientBoostingClassifier

    # Performing Grid Search on a Gradient Boosting Classifier:
    # Commented because I already have the best estimator

    #from sklearn.model_selection import GridSearchCV

    #parameters = {'loss' : ['deviance', 'exponential'],
    #              'learning_rate' : np.geomspace(0.001, 100, 6),
    #              'n_estimators' : np.logspace(2, 6, 6, base=3).astype(int),
    #              'max_depth' : np.arange(2, 6)}
    #parameters

    #gbc = GradientBoostingClassifier()
    #clf = GridSearchCV(gbc, parameters, scoring='roc_auc', n_jobs=-1, cv=5, verbose=1)
    #clf.fit(X_train, y_train)

    #df_cv = pd.DataFrame(clf.cv_results_)
    #estimator = clf.best_estimator_

    estimator = GradientBoostingClassifier(criterion='friedman_mse', init=None,
                  learning_rate=0.01, loss='deviance', max_depth=3,
                  max_features=None, max_leaf_nodes=None,
                  min_impurity_split=1e-07, min_samples_leaf=1,
                  min_samples_split=2, min_weight_fraction_leaf=0.0,
                  n_estimators=302, presort='auto', random_state=None,
                  subsample=1.0, verbose=0, warm_start=False)
    estimator.fit(X_train, y_train)
    probabilities = pd.Series(list(map(lambda x : x[1], estimator.predict_proba(X_test))), index=X_test.index)
    return probabilities
salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [8]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:
def new_connections_predictions():
    # Transforming the attribute 'Department' into 'community', which is used by soundarajan_hopcroft measures
    # There might be a better way of writting the two lines below, but I don't know:
    for node, value in nx.get_node_attributes(G, 'Department').items():
        G.node[node]['community'] = value


    # Creating the measures arrays
    common_neigh = {(e[0], e[1]) : len(list(nx.common_neighbors(G, e[0], e[1]))) for e in nx.non_edges(G)}
    adamic_adar = {(u, v) : ind for u, v, ind in nx.adamic_adar_index(G)}
    preferential_attachment = {(u, v) : ind for u, v, ind in nx.preferential_attachment(G)}
    cnsh = {(u, v) : ind for u, v, ind in nx.cn_soundarajan_hopcroft(G)}
    rash = {(u, v) : ind for u, v, ind in nx.ra_index_soundarajan_hopcroft(G)}

    # Adding each measure into the DataFrame
    future_connections['CommonNeighbors'] = pd.Series(common_neigh)
    future_connections['AdamicAdar'] = pd.Series(adamic_adar)
    future_connections['PreferencialAttachment'] = pd.Series(preferential_attachment)
    future_connections['CNSH'] = pd.Series(cnsh)
    future_connections['RASH'] = pd.Series(rash)

    df_test = future_connections[np.isnan(future_connections['Future Connection'])]
    df_train = future_connections[~np.isnan(future_connections['Future Connection'])]

    X_train = df_train[['CommonNeighbors', 'AdamicAdar', 'PreferencialAttachment', 'CNSH', 'RASH']]
    y_train = df_train['Future Connection']
    X_test = df_test[['CommonNeighbors', 'AdamicAdar', 'PreferencialAttachment', 'CNSH', 'RASH']]


    # Performing Grid Search on a Linear Regression Classifier:
    # Commented because I already have the best estimator

    #from sklearn.model_selection import GridSearchCV

    #parameters = {'penalty' : ['l1', 'l2'],
    #          'C' : np.geomspace(0.01, 10, 4)}
    #print(parameters)

    #lrc = LogisticRegression()
    #clf = GridSearchCV(lrc, parameters, scoring='roc_auc', n_jobs=-1, cv=5, verbose=2)
    #clf.fit(X_train, y_train)

    #estimator = clf.best_estimator_
    #estimator

    from sklearn.linear_model import LogisticRegression
    estimator = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                      intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
                      penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
                      verbose=0, warm_start=False)
    estimator.fit(X_train, y_train)

    probabilities = pd.Series(list(map(lambda x : x[1], estimator.predict_proba(X_test))), index=X_test.index)
    return probabilities
new_connections_predictions()

AttributeError: 'Graph' object has no attribute '_node'